In [ ]:
import os
if os.getcwd().endswith('nb'):
    os.chdir('..')


from lib.pdf_converter import pdf_to_txt_list, pdf_to_img_list

import json
import re
import requests

DATA_DIR = "data"
NAME = "cs685"

save_dir = os.path.join(DATA_DIR, NAME)
url_list_path = os.path.join(save_dir, "urls.txt")

def url_to_file_path(url):
    return os.path.join(save_dir, "pdfs", url.split('/')[-1])

def download_url(url):
    # make save directory
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    file_path = url_to_file_path(url)
    # check if file exists before downloading
    if not os.path.exists(file_path):
        with open(file_path, 'wb') as f:
            print("Downloading {}".format(url))
            f.write(requests.get(url).content)

In [ ]:
# Download the PDF files
urls = []
with open(url_list_path, 'r') as f:
    for line in f:
        urls.append(line.strip())



for url in urls:
    download_url(url)

# 514 PDFS

In [ ]:
# Create a dictionary of PDF texts with keys as the PDF file names and page numbers with text as the values
# text_dict = {
    # page_name: {
    #     "link": "pdf_url#page=page_number",
    #     "text": "text_on_page"
    #     "embedding": [x1, x2, x3, ...]
    # }
# }
text_dict = {}

for url in urls:
    file_path = url_to_file_path(url)
    if not os.path.exists(file_path):
        download_url(url)
    print("Processing {}".format(file_path))
    pages = pdf_to_txt_list(file_path)
    file_name = file_path.split('/')[-1]
    for slide_num, page in enumerate(pages):
        slide_num += 1
        # Use regex to search for "{digits}\n\n\f"
        search_result = re.search(r'\n\n\d+\n\n', page)
        if search_result:
            # Get the page number
            page_num = int(search_result.group(0)[2:-2])
            # Remove the page number from the text
            page = page[:search_result.start()]
        else:
            print("Warning: No page number found for {} slide {}".format(file_name, slide_num))
        
        page_name = f"{file_name[:-4]}-{page_num}"
        link = f"{url}#page={slide_num}"
        
        text_dict[page_name] = {
            "link": link,
            "text": page,
            "file_path": file_path,
            "slide_num": slide_num
        }
    
    slide_nums = []
    keys = []
    for key in text_dict:
        if text_dict[key]["file_path"] == file_path:
            slide_nums.append(text_dict[key]["slide_num"])
            keys.append(key)
    
    img_files = pdf_to_img_list(file_path, slide_nums, os.path.join(save_dir, "imgs"))

    assert len(img_files) == len(slide_nums), "Number of images and slide numbers do not match"

    for key, img_file in zip(keys, img_files):
        text_dict[key]["img"] = img_file

# Save the dictionary to a json file with file name as the directory name
with open(f"data/{NAME}/texts.json", 'w') as f:
    json.dump(text_dict, f, indent=4)

# 685 PDFS

In [5]:
# Create a dictionary of PDF texts with keys as the PDF file names and page numbers with text as the values
# text_dict = {
    # page_name: {
    #     "link": "pdf_url#page=page_number",
    #     "text": "text_on_page"
    #     "embedding": [x1, x2, x3, ...]
    # }
# }
text_dict = {}

for url in urls:
    file_path = url_to_file_path(url)
    if not os.path.exists(file_path):
        download_url(url)
    print("Processing {}".format(file_path))
    pages = pdf_to_txt_list(file_path)
    file_name = file_path.split('/')[-1]
    for slide_num, page in enumerate(pages):
        if len(page) > 1000:
            continue
        slide_num += 1
        page_name = f"{file_name[:-4]}-{slide_num}"
        link = f"{url}#page={slide_num}"
        
        text_dict[page_name] = {
            "link": link,
            "text": page,
            "file_path": file_path,
            "slide_num": slide_num
        }
    
    slide_nums = []
    keys = []
    for key in text_dict:
        if text_dict[key]["file_path"] == file_path:
            slide_nums.append(text_dict[key]["slide_num"])
            keys.append(key)
    
    img_files = pdf_to_img_list(file_path, slide_nums, os.path.join(save_dir, "imgs"))

    assert len(img_files) == len(slide_nums), "Number of images and slide numbers do not match"

    for key, img_file in zip(keys, img_files):
        text_dict[key]["img"] = img_file

# Save the dictionary to a json file with file name as the directory name
with open(f"data/{NAME}/texts.json", 'w') as f:
    json.dump(text_dict, f, indent=4)

Processing data/cs685/pdfs/00-intro.pdf
Processing data/cs685/pdfs/01-lm.pdf
Processing data/cs685/pdfs/02-neural-lms.pdf
Processing data/cs685/pdfs/04-attention.pdf
Processing data/cs685/pdfs/05-transformers.pdf
Processing data/cs685/pdfs/decoding.pdf
Processing data/cs685/pdfs/tokenization.pdf
Processing data/cs685/pdfs/prompt_learning.pdf
Processing data/cs685/pdfs/gen_eval.pdf
Processing data/cs685/pdfs/multilingual.pdf
Processing data/cs685/pdfs/retrieval-lms.pdf
